In [ ]:
# 주소 목록

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time
import urllib.parse

# 크롬 드라이버 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")

# Keywords to search
keywords = [
    "검색어",
]
start_date = "2024.06.01"
end_date = "2024.08.31"

# 웹 드라이버 초기화
driver = webdriver.Chrome(options=chrome_options)

def fetch_search_results(keyword, start_date, end_date):
    base_url = "https://search.hankyung.com/search/news?query="
    results = []
    collected_urls = set()  # 수집된 기사 URL을 저장하는 집합ㅁ
    page = 1

    while True:
        print(f"{keyword} {page} 페이지 수집 중...")
        search_url = base_url + urllib.parse.quote(keyword) + f"&sort=DATE%2FDESC%2CRANK%2FDESC&period=DATE&area=ALL&sdate={start_date}&edate={end_date}&exact=&include=&except=&page={page}"
        driver.get(search_url)
        
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "ul.article li")))
        except Exception as e:
            print(f"페이지 로딩 중 오류 발생: {e}")
            break

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 기사 목록 가져오기
        article_list = soup.find_all("ul", class_="article")
        if not article_list:
            break

        new_articles_found = False  # 루프 시작 시 새로운 기사가 없는 상태로 초기화

        for ul in article_list:
            articles = ul.find_all("li")
            for item in articles:
                try:
                    # 'div.txt_wrap' 내에서 모든 'a' 태그를 찾고 썸네일 여부에 따라 선택
                    txt_wrap = item.find("div", class_="txt_wrap")
                    a_tags = txt_wrap.find_all("a", href=True)

                    if txt_wrap.find("div", class_="thumbnail"):
                        title_tag = a_tags[1] if len(a_tags) > 1 else a_tags[0]
                    else:
                        title_tag = a_tags[0] if a_tags else None

                    title = title_tag.find("em", class_="tit").text.strip() if title_tag and title_tag.find("em", class_="tit") else ""
                    url = title_tag["href"] if title_tag else ""

                    # URL 중복 검사
                    if url and url not in collected_urls:
                        collected_urls.add(url)
                        new_articles_found = True  # 새로운 기사가 발견됨

                        date_element = item.find("span", class_="date_time")
                        date = date_element.text.strip().split(" ")[0] if date_element else ""

                        source_element = item.find("span", class_=None)
                        source = source_element.text.strip() if source_element else ""

                        writer_element = item.find("p", class_="info")
                        writer_text = writer_element.text.strip() if writer_element else ""
                        writer = writer_text.split('|')[-1].strip() if '|' in writer_text else writer_text

                        # 결과 리스트에 추가
                        results.append({
                            'media': '한국경제', 
                            'title': title, 
                            'date': date, 
                            'section': source, 
                            'article_type': '', 
                            'writer': writer, 
                            'page': '', 
                            'url': url, 
                            'content': ''  # 컨텐츠 정보 없음
                        })

                except Exception as e:
                    print(f"기사 처리 중 오류 발생: {e}")

        # 새로운 기사가 없으면 종료
        if not new_articles_found:
            break

        # 다음 페이지로 이동
        page += 1
                          
    return results

def save_to_excel(data, filename):
    df = pd.DataFrame(data)
    df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
    df = df.drop_duplicates(subset=['url'], keep='first')
    df.to_excel(filename, index=False)

all_results = []    

for keyword in keywords:
    results = fetch_search_results(keyword, start_date, end_date)
    all_results.extend(results)

# 결과를 엑셀 파일로 저장
save_to_excel(all_results, f'3한국경제 {start_date}~{end_date}.xlsx')

# 드라이버 종료
driver.quit()


In [ ]:
# 한국경제

import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup

# Selenium 헤드리스 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 1. 출처와 파일명 변수 설정
file_name = '파일명'  # 파일명 설정
base_path = rf'경로'  # 경로 설정

# 2. 임시 파일 경로와 최종 파일 경로 생성
temp_output_path = os.path.join(base_path, f'{file_name}.xlsx')
# 임시 파일 에러를 대비해 본 파일에 덮어쓴다.
output_path = os.path.join(base_path, f'{file_name}_본문.xlsx')  # 최종 파일 경로
error_log_path = os.path.join(base_path, f'{file_name}_error_log.txt')  # 에러 로그 파일 경로

# 3. 파일 경로 생성 및 엑셀 파일 불러오기
input_file = os.path.join(base_path, f'{file_name}.xlsx')
df = pd.read_excel(input_file, engine='openpyxl')

# ChromeDriver 설정 없이 크롬 브라우저 실행
driver = webdriver.Chrome(options=chrome_options)

def wait_for_article_body(wait, class_names, id_names=None):
    """주어진 클래스 또는 ID 이름 중 첫 번째로 매칭된 요소를 반환합니다."""
    
    # 클래스 이름으로 요소 찾기
    for class_name in class_names:
        try:
            return wait.until(EC.presence_of_element_located((By.CLASS_NAME, class_name)))
        except TimeoutException:
            continue  # 다음 클래스 이름으로 시도
    
    # ID 이름으로 요소 찾기 (id_names가 제공된 경우)
    if id_names:
        for id_name in id_names:
            try:
                return wait.until(EC.presence_of_element_located((By.ID, id_name)))
            except TimeoutException:
                continue  # 다음 ID 이름으로 시도
    
    return None  # 매칭된 요소가 없으면 None 반환

# URL에서 본문을 스크래핑하는 함수 정의
def get_content_from_url(url):
    content, subtitle = "", ""  # 기본값 초기화
    try:
        driver.get(url)

        # WebDriverWait 설정
        wait = WebDriverWait(driver, 10)

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

                # 서브타이틀 추출 (본문 외부에서 우선 검색)
        subtitle_element = soup.find("div", class_="summary")
        if subtitle_element:
            subtitle = subtitle_element.get_text(separator="\n", strip=True)
            subtitle_element.decompose()  # 서브 타이틀 제거

        # 기사 본문을 찾기 위한 클래스 이름 리스트
        class_names = [ 'newscon', 'article-body']
        id_names = [ 'newscon', 'article-body', 'articletxt']

        # 첫 번째로 매칭된 요소 찾기
        article_body_element = wait_for_article_body(wait, class_names, id_names)

        if article_body_element:
            # WebElement를 BeautifulSoup 객체로 변환
            article_body_soup = BeautifulSoup(article_body_element.get_attribute('outerHTML'), 'html.parser')
        
            # 서브타이틀 추출 (본문 외부에서 우선 검색)
            subtitle_element = article_body_soup.find("div", class_="summary")
            if subtitle_element:
                subtitle = subtitle_element.get_text(separator="\n", strip=True)
                subtitle_element.decompose()  # 서브 타이틀 제거

            # 'figure' 태그 제거
            for caption in article_body_soup.find_all('figure'):
                caption.decompose()

            # 본문 텍스트 추출
            content = article_body_soup.get_text(separator='\n', strip=True)
        else:
            print(f"본문 요소를 찾을 수 없음 URL: {url}")
        
        if content:
            print(f"본문 추출 성공 URL: {url}")
        else:
            print(f"본문 추출 실패 URL: {url}")

        return content, subtitle

    except Exception as e:
        print(f"Error fetching URL {url}: {e}")
        return content, subtitle  # 예외 발생 시 기본값 반환

# 진행 중인 기사 스크래핑 상태를 트래킹할 리스트와 로그 파일
error_log = []
total_articles = len(df)

# 각 행의 URL에서 본문 내용을 스크랩하여 content 컬럼에 저장합니다.
for index, row in df.iterrows():
    print(f"Processing {index + 1}/{total_articles}: {row['url']}")
    content, subtitle = get_content_from_url(row['url'])
    if content is None:
        error_log.append(row['url'])
    df.at[index, 'content'] = content
    df.at[index, 'sub-title'] = subtitle

        # 100개마다 파일 저장
    if (index + 1) % 100 == 0:
        df.to_excel(temp_output_path, index=False)
        print(f"임시 파일을 저장했습니다: {temp_output_path}")

# 웹드라이버 종료
driver.quit()

# 오류 로그 저장
if error_log:
    with open(error_log_path, 'w') as f:
        for url in error_log:
            f.write(f"{url}\n")

# 변경된 데이터를 파일로 저장합니다.
df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df = df[~df['content'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df.to_excel(output_path, index=False)

print("본문 내용을 성공적으로 스크랩하여 저장했습니다.")      
